In [11]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import geemap
import ee

In [12]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [13]:
def get_data(long, lat, start_date, end_date):
  Map = geemap.Map()
  geometry = ee.Geometry.Point([long,lat])
  image = ee.ImageCollection("COPERNICUS/S2_SR") \
              .filterBounds(geometry) \
              .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',20)) \
              .first()

  ndwi = image.normalizedDifference(['B3', 'B8'])

  ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
  ndwiMasked1= ndwiMasked.toInt()

  vectors = ndwiMasked1.reduceToVectors(scale = 30.0, geometryType = 'polygon', eightConnected = False, maxPixels = 10000000, bestEffort=True)

  Map.addLayer(geometry)
  sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
                .filterDate(start_date,end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                .median()

  visualization = {
  'min': 0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
  }
  Map.addLayer(sentinel, visualization, 'sent2rgb')

  mndwi = sentinel.normalizedDifference(['B3','B11']).rename('mndwi')
  mndwitr = mndwi.gt(0)
  ndsi = sentinel.normalizedDifference(['B11','B12']).rename('ndsi')
  ndsi2 = sentinel.normalizedDifference(['B11','B12']).rename('ndsi2').mask(mndwitr)
  Map.addLayer(ndsi2,{'min':0.1,'max':0.4,'palette':['cyan','orange','red']},'salinity')
  ndti = sentinel.normalizedDifference(['B4','B3']).rename('ndti')
  ndti2 = sentinel.normalizedDifference(['B4','B3']).rename('ndti2').mask(mndwitr)
  Map.addLayer(ndti2,{'min':-1,'max':+1,'palette':['blue','pink','brown']},'turbidity')

  ndci = sentinel.normalizedDifference(['B5','B4']).rename('ndci')
  ndci2 = sentinel.normalizedDifference(['B5','B4']).rename('ndci2').mask(mndwitr)
  Map.addLayer(ndci2,{'min':-1,'max':+1,'palette':['green','pink','brown']},'chlorophyll')


  ph  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')
  ph2  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph2').mask(mndwitr)
  Map.addLayer(ph2,{'min':0,'max':14,'palette':['red','yellow','cyan']},'ph')

  dissolvedoxygen  = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')
  dissolvedoxygen2  = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen2').mask(mndwitr)
  Map.addLayer(dissolvedoxygen2,{'min':6.5,'max':8,'palette':['red','green','blue']},'do')

  nitrate = sentinel.expression('(B4 + B3) / (B4 - B3)', {
      'B3': sentinel.select('B3'),
      'B4': sentinel.select('B4')}).rename('nitrate')
  Map.addLayer(nitrate,{'min':-1,'max':1,'palette':['blue','white','green']},'Nitrate')

  phosphate = sentinel.expression('(B4 - B5) / (B4 + B5)', {
      'B4': sentinel.select('B4'),
      'B5': sentinel.select('B5')
  }).rename('phosphate')
  Map.addLayer(phosphate,{'min':-1,'max':1,'palette':['brown','white','green']},'Phosphate')



  col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate(start_date,end_date) \
  .filterBounds(vectors).median()

  temp  = col.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temp')


  starting = start_date
  ending = end_date

  data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(starting, ending).filterBounds(vectors)

  rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
                .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(vectors)
  dm_2021_Jan_August_test = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
  dom2 = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).mask(mndwitr)
  Map.addLayer(dom2,{'min':0,'max':0.8,'palette':['green','red','yellow']},'Dissolved organic matter')
  suspended_matter_2021_Jan_August_test= rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')
  suspended_matter2 = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).mask(mndwitr)
  Map.addLayer(suspended_matter2,{'min':0,'max':0.8,'palette':['green','red','yellow']},'suspended_matter')

  latlon = ee.Image.pixelLonLat().addBands(dm_2021_Jan_August_test)

  latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100,
    tileScale = 16)

  data_dom_2021_Jan_August_test = np.array((ee.Array(latlon.get("dom")).getInfo()))

  latlon = ee.Image.pixelLonLat().addBands(suspended_matter_2021_Jan_August_test)
  latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100,
    tileScale = 16)

  data_sm_2021_Jan_August_test= np.array((ee.Array(latlon.get("suspended_matter")).getInfo()))

  latlon = ee.Image.pixelLonLat().addBands(temp)


  latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)

  data_lst = np.array((ee.Array(latlon.get("temp")).getInfo()))


  latlon = ee.Image.pixelLonLat().addBands(ndti)
  latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
  data_ndti = np.array((ee.Array(latlon.get("ndti")).getInfo()))

  latlon = ee.Image.pixelLonLat().addBands(ndsi)
  latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
  data_ndsi = np.array((ee.Array(latlon.get("ndsi")).getInfo()))

  latlon = ee.Image.pixelLonLat().addBands(ndci)
  latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
  data_ndci = np.array((ee.Array(latlon.get("ndci")).getInfo()))

  latlon = ee.Image.pixelLonLat().addBands(dissolvedoxygen)

  latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100,
    tileScale = 16)

  data_do = np.array((ee.Array(latlon.get("dissolvedoxygen")).getInfo()))

  latlon = ee.Image.pixelLonLat().addBands(ph)

  latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
  data_ph = np.array((ee.Array(latlon.get("ph")).getInfo()))
 

  df = pd.concat([pd.DataFrame(data_do, columns = ['Dissolved Oxygen']),\
             pd.DataFrame(data_ndsi, columns = ['Salinity']),\
             pd.DataFrame(data_lst, columns = ['Temperature']),\
             pd.DataFrame(data_ph, columns = ['pH']),\
             pd.DataFrame(data_ndti, columns = ['Turbidity']),\
             pd.DataFrame(data_dom_2021_Jan_August_test, columns = ['Dissolved Organic Matter']),\
             pd.DataFrame(data_sm_2021_Jan_August_test, columns = ['Suspended Matter']),\
             pd.DataFrame(data_ndci, columns = ['Chlorophyll'])], axis=1,sort=False)

  return df


In [14]:
def send_df(df2):
    df2 = df2.dropna()
    df2['Dissolved Organic Matter'] = df2['Dissolved Organic Matter']*1000
    df2['Suspended Matter'] = df2['Suspended Matter']*1000
    test = pd.DataFrame(MinMaxScaler().fit_transform(df2.drop(
        ['Salinity'], axis=1)), columns=df2.drop(['Salinity'], axis=1).columns)
    return df2, test


lat, lon = 23.204596898570106, 77.42153953958919
start_date, end_date = '2021-01-01', '2021-08-31'


In [15]:
df = get_data(lon, lat, start_date, end_date)

In [16]:
df.head()

,Dissolved Oxygen,Salinity,Temperature,pH,Turbidity,Dissolved Organic Matter,Suspended Matter,Chlorophyll
0,3.06730,0.128733,33.912917,7.933939,-0.093764,0.473920,0.590636,0.107739
1,6.43490,0.176895,34.302571,8.019467,-0.077675,0.491333,0.609152,0.149591
2,5.87110,0.220141,33.581369,8.056200,-0.124529,0.492745,0.600119,0.170916
3,6.15700,0.095065,36.428580,8.014619,-0.104173,0.493983,0.601424,0.080556
4,6.21285,0.072193,36.267933,8.003901,-0.071184,0.493274,0.602325,0.067619


In [17]:
df.to_csv('df_jbul.csv')